In [3]:
import numpy as np
import pandas as pd
import re
#import time
from datasketch import MinHash, MinHashLSHForest

In [4]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [5]:
def get_forest(data, perms):
    #start_time = time.time()
    
    minhash = []
    
    for text in data['text']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    #print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [6]:
def predict(text, database, perms, num_results, forest):
    #start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]
    
    #print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [7]:
db = pd.read_csv('Final_Reformatted_Clean_Corpus.csv')
permutations = 128
# db=corpus
db['text'] = db['Headline'] + ' ' + db['Entire_News']
forest = get_forest(db, permutations)

In [8]:
pd.set_option('display.max_colwidth', -1) 

<ipython-input-8-b7cc634e6674>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [9]:
num_recommendations = 10
title = db['Headline'][20225]
result = predict(title, db, permutations, num_recommendations, forest)
display('Querry Headline', db['Headline'][20225])
display('Top Recommendation(s) is(are)', result)

'Querry Headline'

"Jio To Launch A Low-Cost Laptop Called The 'JioBook' With 4G LTE Connectivity"

'Top Recommendation(s) is(are)'

,Datetime,Category,Subcategory,Headline,Summary,Entire_News,Author,News_Link,text
7201,2017-06-12 21:26:00,Technology,games,The Evil Within 2 announcement trailer,"Resident Evil creator Shinji Mikami returns with the follow-up to his latest horror IP The Evil Within, which was revealed by publisher Bethesda during its E3 2017 press conference.","Resident Evil creator Shinji Mikami returns with the follow-up to his latest horror IP The Evil Within, which was revealed by publisher Bethesda during its E3 2017 press conference.",admin,https://www.ibtimes.co.in/the-evil-within-2-announcement-trailer-730516,"The Evil Within 2 announcement trailer Resident Evil creator Shinji Mikami returns with the follow-up to his latest horror IP The Evil Within, which was revealed by publisher Bethesda during its E3 2017 press conference."
7169,2017-07-22 14:30:00,Technology,games,"Video game news round-up: Doomfist release, Telltale Games and a legendary Pokemon Go announcement","Your pick of the biggest gaming stories from the past seven days, this week including Overwatch, Telltale Games and Pokemon Go.","Your pick of the biggest gaming stories from the past seven days, this week including Overwatch, Telltale Games and Pokemon Go.",admin,https://www.ibtimes.co.in/video-game-news-round-up-doomfist-release-telltale-games-and-a-legendary-pokemon-go-announcement-735643,"Video game news round-up: Doomfist release, Telltale Games and a legendary Pokemon Go announcement Your pick of the biggest gaming stories from the past seven days, this week including Overwatch, Telltale Games and Pokemon Go."
15916,2021-03-09 18:49:00,World,Uk-news,Karachi Activists Have Mixed Feelings On Meghan Intv,"There were mixed reactions in Pakistan following a television interview with the UK's Prince Harry and Meghan Markle, in which the couple said they encountered racist attitudes and a lack of support that drove the Duchess to thoughts of suicide.","There were mixed reactions in Pakistan following a television interview with the UK's Prince Harry and Meghan Markle, in which the couple said they encountered racist attitudes and a lack of support that drove the Duchess to thoughts of suicide.Some women interviewed in Karachi who had gathered to march on International Women's day said not much would change following the explosive interview, while others said the reputation of Britain's Royal Family could be affected.In the two hour interview aired in the US on Sunday night, the couple painted a deeply unflattering picture of the life inside the royal household. (Disclaimer: This story has not been edited by www.republicworld.com and is auto-generated from a syndicated feed.)",Associated press television news,https://www.republicworld.com/world-news/uk-news/karachi-activists-have-mixed-feelings-on-meghan-intv.html,"Karachi Activists Have Mixed Feelings On Meghan Intv There were mixed reactions in Pakistan following a television interview with the UK's Prince Harry and Meghan Markle, in which the couple said they encountered racist attitudes and a lack of support that drove the Duchess to thoughts of suicide.Some women interviewed in Karachi who had gathered to march on International Women's day said not much would change following the explosive interview, while others said the reputation of Britain's Royal Family could be affected.In the two hour interview aired in the US on Sunday night, the couple painted a deeply unflattering picture of the life inside the royal household. (Disclaimer: This story has not been edited by www.republicworld.com and is auto-generated from a syndicated feed.)"
6771,2017-06-09 15:51:00,Technology,internet,Who are the DUP?,"The U.K. election has resulted in a hung parliament, and Conservatives are understood to be in talks with the Democratic Unionist Party. The DUP is the largest unionist political party in Northern Ireland and was founded in 1971, with the main political aim of keeping Northern Ireland part of the U.K.","The U.K. election ha

In [8]:
num_recommendations = 10
title = db['Entire_News'][20225]
result = predict(title, db, permutations, num_recommendations, forest)
display('Querry Headline', db['Headline'][20225])
display('Top Recommendation(s) is(are)', result)

'Querry Headline'

"Jio To Launch A Low-Cost Laptop Called The 'JioBook' With 4G LTE Connectivity"

'Top Recommendation(s) is(are)'

20225    Jio To Launch A Low-Cost Laptop Called The 'JioBook' With 4G LTE Connectivity                            
11334    25 years after Amul topical on Urmila Matondkar in Rangeela, divided India is outraged                   
10798    Haters, stop all your hate, the prices of Redmi K20 and Redmi K20 Pro are fine                           
527      A loss that keeps feet on ground                                                                         
10865    Best smartphones with 6000mAh battery to buy this February: Poco M3 joins Asus ROG Phone 3, Redmi 9 Power
2067     Apple iPhone 12 Pro Max review: The real Pro phone                                                       
10517    Poco M3 review: The smartphone to beat this year under Rs 12,000                                         
2134     MacBook Air (2020) review: Nobody does it better                                                         
2427     Redmi Note 9 Pro 5G vs Redmi Note 9 Pro: What has changed with the 5G v